In [ ]:
import pandas as pd
import json
import os
import numpy as np

In [ ]:
json_path = "/mnt/d/code/Balu_CARE/datasets/20250523_192I_denoising_5to40F.json"
image_index_csv = "/mnt/d/code/Balu_CARE/datasets/raw_image_index.csv"

In [ ]:
image_index = pd.read_csv(image_index_csv)

In [ ]:
image_index.head()

In [ ]:
dataset = json.load(open(json_path,'r'))

In [ ]:
ids = dataset['FLAME_Dataset']['image_ids']

In [ ]:
image_index.loc[image_index['id']==6]['image'].values[0]

In [ ]:
# image_paths = {}
# for id in ids:
#     dirname = os.path.dirname(image_index.loc[image_index['id'] == id]['image'].values[0])
#     if dirname not in image_paths.keys():
#         image_paths[dirname] = 1
#     else:
#         image_paths[dirname] += 1

In [ ]:
# print("\n".join(image_paths.keys()))

In [ ]:
# np.sum(list(image_paths.values()))

In [ ]:
# image_paths

In [ ]:
import tifffile as tiff
from flame import FLAMEImage
from flame.utils import min_max_norm
from matplotlib import pyplot as plt

In [ ]:
path = "/mnt/d/data/raw/S276_250521_CAREtraining_PL/SCCdogear_ExVivo_neck/Mosaic07_2x2_FOV600_z85_3Ch/Im_00002.tif"

In [ ]:
test = FLAMEImage(path, "tileData.txt")

In [ ]:
test.openImage()

In [ ]:
imageData = test.imageData.copy()

In [ ]:
imageData.shape

In [ ]:
imageData.min()

In [ ]:
imageData.max()

In [ ]:
ds = json.load(open("/mnt/d/code/Balu_CARE/datasets/20250527_112I_denoising_5to40F.json", 'r'))

In [ ]:
pct1 = np.array(ds["FLAME_Dataset"]["input"]['pixel_1pct'])
pct99 = np.array(ds["FLAME_Dataset"]["input"]['pixel_99pct'])

In [ ]:
mmn = min_max_norm(imageData, pct1, pct99)

In [ ]:
mmn.min()

In [ ]:
mmn.max()

In [ ]:
test.get_frames(0, 5)

In [ ]:
output_test = tiff.imread("/mnt/d/data/processed/20250527_112I_denoising_5to40F/train/id127_frames40.tif")

In [ ]:
output_test.all() == test.get_frames(0, 40).all()